In [1]:
#https://github.com/amazon-science/efficient-longdoc-classification
from functools import partial
import nltk
import pickle as pk
import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_bart import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed_everything(1376)

/home/nlplab/anaconda3/envs/development/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [2]:
z=torch.ones((1,2),dtype=torch.long,device='cuda')
z.device

device(type='cuda', index=0)

In [13]:
configs

Namespace(is_baseline=False, model_base='facebook/bart-base', output_dir='trained_models_sum_boundary/', task_type='xsum', run_id='bart_base_model_context_enforcer_250', eval_steps=1000, learning_rate=5e-05, max_seq_len=800, evaluation_strategy='epoch', save_strategy='epoch', seed=10, lr_scheduler_type='linear', weight_decay=0.0, warmup_ratio=0.25, num_train_epochs=10, save_total_limit=1, per_device_train_batch_size=12, per_device_eval_batch_size=12, gradient_accumulation_steps=4, fp16=True, verbose=False, context_max_len=250)

In [4]:
configs= pk.load(open("trained_models_sum_boundary/train_args.ap",'rb'))
context_max_len=configs.context_max_len
context_max_len_list = [context_max_len]#,300,450]
context_sampling_bounds=(0.1, 0.45)

In [5]:
configs.model_base

'facebook/bart-base'

In [6]:
tokenizer = create_text_tokenizer(configs.model_base)

task_dataset_gen = get_dataset_specified_tasks(configs.task_type)

train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs.model_base,
        context_max_len = context_max_len,
        context_sampling_bounds = context_sampling_bounds,
        context_max_len_list= context_max_len_list,#is_baseline=True
    )

Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
100%|██████████| 3/3 [00:00<00:00, 472.72it/s]


In [7]:
generator = model_builder()
train_model_path = "trained_models_sum_boundary/bart_base_model_context_enforcer/checkpoint-25506/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Some weights of BartForContextualRecovery were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.layers.4.context_enforcer_layer_norm.bias', 'encoder.layers.3.context_enforcer._wrc.weight', 'encoder.layers.5.context_enforcer._wrc.bias', 'encoder.layers.2.context_enforcer._left_mha.out_proj.bias', 'encoder.layers.5.context_enforcer._wlc.bias', 'encoder.layers.2.context_enforcer._left_mha.in_proj_bias', 'encoder.layers.2.context_enforcer._wlc.bias', 'encoder.layers.2.context_enforcer._wlc.weight', 'encoder.layers.3.context_enforcer._right_mha.out_proj.weight', 'encoder.layers.5.context_enforcer._right_mha.in_proj_bias', 'encoder.layers.3.context_enforcer_layer_norm.bias', 'encoder.layers.5.context_enforcer._right_mha.out_proj.weight', 'encoder.layers.5.context_enforcer._left_mha.in_proj_weight', 'encoder.layers.4.context_enforcer._right_mha.in_proj_bias', 'encoder.layers.4.context_enforcer._left_mha.in_proj_bias', 'encoder.layers.4.conte

In [15]:
te1= test_dataset[16]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [30]:
example= {'document':"""
           The character Buzz Watson first appeared in the second episode of the first season, "About Face". 
           Originally a recurring role, as the series progressed the character's visibility increased until Buzz became one of the central characters. 
           In the season 1 episode "Standards and Practices", it was revealed that he speaks fluent Spanish, which he learned from his Mexican stepfather.
           In the season 7 episode "You Have the Right to Remain Jolly", it was revealed he had a sister named Casey, who was played by Christine Woods.
           """}
te1=test_dataset._process_data(example)
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [33]:
seed_everything(1376)
boundary_sample =  (0.15, 0.5)
seq_len = te1.input_ids[:1024].shape[0]

boundary_start = int(0.45*seq_len) 
context_boundary = compute_context_boundary(seq_len,context_max_len=context_max_len)
context_boundary

Global seed set to 1376


(24, 137)

In [1]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=context_boundary,
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=169,
        early_stopping=True,
        use_cache=True,
        num_beams=8,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

NameError: name 'generator' is not defined

In [20]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

'Limiting access to federal research would do an "enormous disservice" to the US and the world according to former Nasa chief scientist.'

In [51]:
te1.input_ids.shape

torch.Size([1187])

In [21]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

Dr Waleed Abdalati told the BBC the that continued access to data is in "everyone's best interest".
Many US scientists are rushing to copy information onto servers outside the control of the federal government.
They are afraid the Trump administration will curb access to climate and other research.
The President-elect has blown hot and cold on the issue of climate change, having previously tweeted about global warming being a hoax.
On Wednesday, one of his advisers compared scientists who support the mainstream view on global warming to flat-Earthers.
"There was an overwhelming science that the Earth was flat and there was an overwhelming science that we were the centre of the world," said Anthony Scaramucci, a member of the Trump transition committee, on CNN.
"We get a lot of things wrong in the scientific community."
Now at the Co-operative Institute for Research in Environmental Sciences, Dr Abdalati served as Nasa's chief scientist in 2011, for two years.
He says it is too early to

In [22]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800

In [29]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
from context_enforcement.data.common import write_to_file
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])
def generate():
    test_data_loader = DataLoader(test_dataset,batch_size=12,
                                sampler= SequentialSampler(test_dataset),
                                collate_fn= SmartCollator(
                pad_token_id=train_dataset.tokenizer.pad_token_id,
                max_len=configs.max_seq_len,
                context_max_len=context_max_len,
                context_sampling_bounds=context_sampling_bounds,
            
            ))

    output_summaries =[]
    for batch in tqdm.tqdm(test_data_loader):
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        
        seq_len = b_input_ids.shape[1]

        context_boundary = compute_context_boundary(seq_len,
                                                    context_max_len=110)
        
        boundary_mask =  batch.get("boundary",)
        bb=generator.generate(input_ids=b_input_ids,
                attention_mask=b_input_mask,
                #context_boundary=boundary_mask,
                num_beams=10,
                do_sample=False,
                num_return_sequences=1,
                max_length=140)
        sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
        output_summaries+=sentences
    return output_summaries

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [28]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

In [30]:
context_lens = [200,720,650,350,100]
outputs = {}
results = {}
for cl in context_lens:
    print(f'Generating for the context length: {cl}')
    generator.model.context_max_len= cl
    generator.model.encoder.context_max_len= cl
    rbase_output = generate()
    outputs[cl] = rbase_output
    
    write_to_file(rbase_output[:len(test_dataset)], 
              f"outputs/multi_context/best_base_final_{cl}")
    
    scores = metrics.compute(predictions=rbase_output,references=targets)
    print(scores)
    
    results[cl]= scores
    
    
    

Generating for the context length: 200


100%|██████████| 945/945 [08:19<00:00,  1.89it/s]


Done
{'bleu': 0.12346216754890589, 'precisions': [0.4375689199934979, 0.1776036248468456, 0.09612053612814771, 0.05588868746029532], 'brevity_penalty': 0.863722634039824, 'length_ratio': 0.8722170704229669, 'translation_length': 227619, 'reference_length': 260966, 'meteor': 0.35359416748473077, 'rouge1': 0.40248422610308543, 'rouge2': 0.18328221126363148, 'rougeL': 0.33009269996078755, 'rougeLsum': 0.3300614517511684}
Generating for the context length: 720


100%|██████████| 945/945 [10:18<00:00,  1.53it/s]


Done
{'bleu': 0.1257888249451646, 'precisions': [0.4377841740444545, 0.1791020085562382, 0.09760767869694839, 0.05722976542823581], 'brevity_penalty': 0.8695117278704764, 'length_ratio': 0.8773288474360644, 'translation_length': 228953, 'reference_length': 260966, 'meteor': 0.3565514119827842, 'rouge1': 0.4047119203068985, 'rouge2': 0.185856595769887, 'rougeL': 0.332006465804801, 'rougeLsum': 0.33195165056651943}
Generating for the context length: 650


...

Done
{'bleu': 0.1256403068872618, 'precisions': [0.4381767948566864, 0.17878675170700573, 0.09728983506502943, 0.05701369989390655], 'brevity_penalty': 0.8702041264108997, 'length_ratio': 0.8779419541242921, 'translation_length': 229113, 'reference_length': 260966, 'meteor': 0.3563541597980937, 'rouge1': 0.4050806838364527, 'rouge2': 0.18561721029291084, 'rougeL': 0.3319262324656158, 'rougeLsum': 0.33192334002381724}
Generating for the context length: 350


100%|██████████| 945/945 [09:11<00:00,  1.71it/s]


Done
{'bleu': 0.1257626122759007, 'precisions': [0.4382902891885163, 0.17890146449955657, 0.09765710213638545, 0.05715340935321546], 'brevity_penalty': 0.8695030702543955, 'length_ratio': 0.8773211836024616, 'translation_length': 228951, 'reference_length': 260966, 'meteor': 0.3562012415308598, 'rouge1': 0.4047128146167146, 'rouge2': 0.18550529759109602, 'rougeL': 0.33186920040252776, 'rougeLsum': 0.3318646742987025}
Generating for the context length: 100


100%|██████████| 945/945 [07:49<00:00,  2.01it/s]


Done
{'bleu': 0.11964712510107044, 'precisions': [0.4326436377229051, 0.17240038692776577, 0.0924841374122612, 0.05362138236647655], 'brevity_penalty': 0.8627477081454109, 'length_ratio': 0.8713587210594483, 'translation_length': 227395, 'reference_length': 260966, 'meteor': 0.3474841689817138, 'rouge1': 0.39724258276498764, 'rouge2': 0.17816489836175736, 'rougeL': 0.32460150343422817, 'rougeLsum': 0.32467930362884756}


In [11]:
base_output = generate()

100%|██████████| 945/945 [08:28<00:00,  1.86it/s]


In [17]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800
base_output8 = generate()

100%|██████████| 945/945 [10:04<00:00,  1.56it/s]


In [18]:
generator.model.context_max_len= 600
generator.model.encoder.context_max_len= 600
base_output6 = generate()

100%|██████████| 945/945 [10:02<00:00,  1.57it/s]


In [19]:
generator.model.context_max_len= 450
generator.model.encoder.context_max_len= 450
base_output45 = generate()

100%|██████████| 945/945 [09:30<00:00,  1.66it/s]


In [10]:
#os.makedirs("outputs/multi_context/")

In [25]:
#context_enforcement/data/common.py

write_to_file(base_output8[:len(test_dataset)], 
              "outputs/multi_context/best_base_final_800")

Done


In [14]:
targets[690],base_output[690]#,base_output_8[690],base_output_45[690]#output_summaries2[690],output_summaries[690],output_summaries6[690],output_summaries8[690]

('Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.',
 "Uganda's Koffi Olomide has been arrested in the Democratic Republic of Congo over an alleged assault on one of his dancers.")

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [27]:
scores = metrics.compute(predictions=base_output8,references=targets)
scores

{'bleu': 0.12420563480222273,
 'precisions': [0.4381499894596304,
  0.17908874941070982,
  0.09682579940300837,
  0.05619688787468894],
 'brevity_penalty': 0.864057575751863,
 'length_ratio': 0.8725121280166765,
 'translation_length': 227696,
 'reference_length': 260966,
 'meteor': 0.35420455825818364,
 'rouge1': 0.4026762188767841,
 'rouge2': 0.18481796867562256,
 'rougeL': 0.3305624212280963,
 'rougeLsum': 0.3305498661301927}

In [22]:
scores45 = metrics.compute(predictions=base_output45,
                           references=targets)
scores45

{'bleu': 0.12529167445320155,
 'precisions': [0.43767581217045437,
  0.1782170041154529,
  0.0969340592861464,
  0.056633434208437666],
 'brevity_penalty': 0.8709825773070866,
 'length_ratio': 0.8786316991485481,
 'translation_length': 229293,
 'reference_length': 260966,
 'meteor': 0.35545882674212,
 'rouge1': 0.4042558166707959,
 'rouge2': 0.18471768845866826,
 'rougeL': 0.3318399755379066,
 'rougeLsum': 0.33182453398530987}

In [36]:
{'bleu': 0.13001522827171397,
 'precisions': [0.43264780948765,
  0.17764752271710285,
  0.09726789720912225,
  0.057229256856921935],
 'brevity_penalty': 0.90401138133862,
 'length_ratio': 0.9083367181931746,
 'translation_length': 237045,
 'reference_length': 260966,
 'meteor': 0.3623311725177827,
 'rouge1': 0.40646161409356885,
 'rouge2': 0.18746257090958113,
 'rougeL': 0.3322105156464427,
 'rougeLsum': 20.3334181119671774}

{'bleu': 0.1242693218959997,
 'precisions': [0.4397112710255618,
  0.17984347696769157,
  0.09751670411397062,
  0.05682880711450953],
 'brevity_penalty': 0.8588878671603885,
 'length_ratio': 0.8679674746901895,
 'translation_length': 226510,
 'reference_length': 260966,
 'meteor': 0.3532861459937222,
 'rouge1': 0.4033676736623957,
 'rouge2': 0.18549005001850066,
 'rougeL': 0.3313132654462809,
 'rougeLsum': 0.3313118204210047}

In [17]:
scores

{'bleu': 0.12215108883184916,
 'precisions': [0.42916968255609556,
  0.17216560221044527,
  0.0922525258312802,
  0.0537097051014124],
 'brevity_penalty': 0.8830712157727976,
 'length_ratio': 0.8894032172773465,
 'translation_length': 232104,
 'reference_length': 260966,
 'meteor': 0.35137424074129014,
 'rouge1': 0.398632967632497,
 'rouge2': 0.1799538886903405,
 'rougeL': 0.32609557084990337,
 'rougeLsum': 0.32622241066447516}

In [24]:
scores4

{'bleu': 0.12300035067772054,
 'precisions': [0.4394282548476454,
  0.1790089177800281,
  0.09666579620313663,
  0.056324136455435936],
 'brevity_penalty': 0.8550152182505472,
 'length_ratio': 0.8645762283209307,
 'translation_length': 225625,
 'reference_length': 260966,
 'meteor': 0.35146299353791816,
 'rouge1': 0.4017681486403819,
 'rouge2': 0.18375560820687226,
 'rougeL': 0.32963068346151203,
 'rougeLsum': 0.3296792225053877}

In [18]:
scores5

{'bleu': 0.12202486703008819,
 'precisions': [0.43851006174759005,
  0.17777018935708405,
  0.09596328825046187,
  0.05604684262301944],
 'brevity_penalty': 0.8527557381282802,
 'length_ratio': 0.8626027911681982,
 'translation_length': 225110,
 'reference_length': 260966,
 'meteor': 0.3498951876777049,
 'rouge1': 0.40069971264338244,
 'rouge2': 0.18265325890345102,
 'rougeL': 0.3290151413411355,
 'rougeLsum': 0.3290728012119742}